# Nhap cac thu vien

In [208]:
import pandas as pd
import numpy as np

# Nhap cac phan tu dau vao

In [209]:
""" dict_columns = {
    'x1':[-1, -1, -1, 1],
    'x2':[-3, -1, 1, 2],
    'w1':[0, 1, 0, 0],
    'w2':[0, 0, 1, 0],
    'w3':[0, 0, 0, 1],
    'HeSo':[0, -3, -1, 4]
} """

dict_columns = {
    'x1':[-2, -1, 2],
    'x2':[6, -1, -1],
    'x3':[0, -1, 1],
    'w1':[0, 1, 0],
    'w2':[0, 0, 1],
    'HeSo':[0, -2, 1]
}

""" dict_columns = {
    'x1':[-1, -1, -1, 1],
    'x2':[-3, -1, 1, 2],
    'w1':[0, 1, 0, 0],
    'w2':[0, 0, 1, 0],
    'w3':[0, 0, 0, 1],
    'HeSo':[0, -3, -1, 2]
} """

data = pd.DataFrame(dict_columns)
data

,x1,x2,x3,w1,w2,HeSo
0,-2,6,0,0,0,0
1,-1,-1,-1,1,0,-2
2,2,-1,1,0,1,1


# Dua ve bai toan bo tro

In [210]:
# Ham xay dung bai toan bo tro dua tren bai toan goc
def SupplementaryMath(data, cols_data, rows_data):
    df = data.copy()
    # Cho he so cua cac phan tu o ham muc tieu bang 0
    for i in range(0, cols_data):
        df.iloc[[0], [i]] = 0
        
    # Tao cot x0 
    x0_values = [1] # Cho he so cua cac phan tu dau tien bang 1
    for i in range(1, rows_data): # Cho he so cua cac phan tu con lai bang -1
        x0_values.append(-1) 
    
    # Them cot x0 vao data frame
    df.insert(cols_data-1, 'x0', x0_values)
    
    return df

# Pha 1:

In [211]:
# Ham tim vi tri cua phan tu x0
def Find_x0_Index(df):
    features_name = list(df.columns)
    x0_index = features_name.index('x0')
    return x0_index

In [212]:
# Ham chuyen -0.0 -> +0.0
def ConvertNegativeZero(matrix):
    matrix += 0.0

In [213]:
# Ham tim phan tu cua bien vao
def FindInputIndex(array, cols, rows):
    
    min = array[0]
    min_index = 0
    for i in range(1, cols):
        if array[i] < min:
            min = array[i]
            min_index = i
        
    return min_index

In [214]:
# Ham xoay
def Rotate(matrix, cols, rows, index):
    # Tim bien ra
    min = matrix[1][-1]/abs(matrix[1][index])
    row_min_index = 1

    for i in range(2, rows):
        # Dung bang nen kiem tra cac bien duong de xet bien ra co so
        if matrix[i][index] > 0:
            if matrix[i][-1]/abs(matrix[i][index]) < min:
                min = matrix[i][-1]/abs(matrix[i][index])
                row_min_index = i

    for i in range(0, rows):
        # Neu i dang xet khong la dong chua bien co gia tri nho nhat va gia tri cua bien khac 0
        if i != row_min_index and matrix[i][index] != 0:
            coef = matrix[i][index]/matrix[row_min_index][index]
            matrix[i][:] -= coef * matrix[row_min_index][:]
        # Neu i dang la dong chua bien co gia tri nho nhat va gia tri cua bien khac 0
        elif i == row_min_index and matrix[i][index] != 0:
            matrix[i][:] /= matrix[i][index]
            
    ConvertNegativeZero(matrix)
    
    return matrix        

In [215]:
# Ham kiem tra cac dieu kien
def CheckTheConditions(df, matrix, cols, rows, x0_index):
    # Xoay lan dau tien doi voi x0 > 0
    matrix = Rotate(matrix, cols, rows, x0_index)
    
    while True:
        # Neu cac phan tu khac deu co he so bang 0 va x0 quay tro lai la bien chua co so
        if np.all(matrix[0][:-2] == 0) == True and matrix[0][-1] == 0 and matrix[0][x0_index] == 1:
            print('Switch to Phase 2.') # Chuyen qua pha 2
            check = 2
            break
        # Neu cac phan tu khac deu co he so lon hon hoac bang 0 va x0 khong quay tro lai la bien co so
        elif np.all(matrix[0][:-2] >= 0) == True and matrix[0][x0_index] == 0:
            print('No solution.') # Vo nghiem
            check = 0
            break
        # Truong hop con lai tim bien vao co so, tiep tuc xoay
        else:
            epsilon = matrix[0][:-2]
            input_index = FindInputIndex(epsilon, cols-2, rows)
            matrix = Rotate(matrix, cols, rows, input_index)
            
    return (matrix, check)

### Tim nhung bien co so

In [216]:
# Ham tim nhung bien co so
def FindBasicVariables(matrix, cols, rows):
    # Tao danh sach chua vi tri cua bien co so trong cot va dong chua bien co so
    basic_variables_index = []
    
    for i in range(0, cols-1):
        c0 = matrix[:, i] == 0
        count0 = np.count_nonzero(c0 == True)
        c1 = matrix[:, i] == 1
        count1 = np.count_nonzero(c1 == True)
        
        # Bien co so la nhung bien ma cot cua no chua 1 gia tri 1 va cac gia tri con lai bang 0
        if count0 + count1 == rows:
            row_index = np.where(c1 == True)
            basic_variables_index.append((i, row_index[0][0]))
            
    return basic_variables_index

### Dua ve bai toan chap nhan duoc

In [217]:
# Ham tao ra z moi
def New_z(data, matrix_data, matrix, cols_data, rows_data, cols, rows):
    # Lay cac he so cua ham muc tieu o bai toan goc va bai toan bo tro
    z = matrix_data[0][:]
    z_new = matrix[0][:]
    
    # Tim cac bien co so sau khi giai bai toan bo tro
    basic_variables = FindBasicVariables(matrix, cols, rows)
    bv_index = [] # Tao danh sach chua vi tri cua cac bien co so trong moi cot
    bv_rows = [] # Tao danh sach chua dong chua bien co so trong mot cot
    # Them cac bien vi tri cua cac bien co so trong moi cot va dong cua no vao hai danh sach
    for i in range(0, len(basic_variables)):
        bv_index.append(basic_variables[i][0])
        bv_rows.append(basic_variables[i][1])
    
    z_basic_variables_index = [] # Tao danh sach chua vi tri cua cac bien co so trong moi cot
    z_non_basic_variables_index = [] # Tao danh sach chua vi tri cua cac bien khong co so trong moi cot
    # Tien hanh them cac bien nay vao hai danh sach
    for i in range(0, cols_data-1):
        if i in bv_index and z[i] != 0: # Kiem tra neu bien nay la bien co so va he so cua no o ham muc tieu goc khac 0
            z_basic_variables_index.append(i)
        elif i not in bv_index and z[i] != 0: # Kiem tra neu bien nay la bien khong co so va he so cua no o ham muc tieu goc khac 0
            z_non_basic_variables_index.append(i)
    
    # Chuyen doi z ve z_new ung voi bai toan chap nhan duoc
    for i in range(0, cols-1):
        # Neu i dang xet la bien co so o bai toan bo tro
        if i in z_basic_variables_index:
            temp_i = bv_rows[i]
            temp = matrix[temp_i][:].copy()
            temp[i] = 0
            temp[i+1:] *= -1
            z_new += z[i] * temp
        # Neu i dang xet la bien khong co so o bai toan bo tro
        elif i in z_non_basic_variables_index:
            temp = np.zeros(cols_data)
            temp[i] = 1
            z_new += z[i] * temp
    
    return z_new

In [218]:
# Ham tao ra bang chap nhan duoc
def AcceptableTable(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows):
    # Tim z_new va them z_new vao ma tran
    z_new = New_z(data, matrix_data, matrix, cols_data, rows_data, cols, rows)
    matrix[0][:] = z_new

    ConvertNegativeZero(matrix)

    # Thay doi noi dung cho data frame goc
    features_name = list(df.columns)
    df = pd.DataFrame(matrix, columns=features_name)
        
    return df # Tra ve data frame chap nhan duoc

### Thuc thi pha 1

In [219]:
# Ham thuc thi pha 1
def Phase1(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows, x0_index):
    (matrix, check) = CheckTheConditions(df, matrix, cols, rows, x0_index)
    if check == 2:
        features_name = list(df.columns)
        df = pd.DataFrame(matrix, columns=features_name)
        df = df.drop(columns='x0')
        matrix = np.delete(matrix, np.s_[x0_index:x0_index+1], axis=1) 
        
        df = AcceptableTable(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows)
        return df

In [220]:
#Lưu các thông số dòng và cột của data
rows_data, cols_data = data.shape
matrix_data = np.array(data, dtype=float)
#Đưa về bài toán bổ trợ
df = SupplementaryMath(data, cols_data, rows_data)
#Lưu các thông số dòng và cột của df
rows, cols = df.shape
matrix = np.array(df, dtype=float)

#Pha 1
x0_index = Find_x0_Index(df)
output = Phase1(data, matrix_data, df, matrix, cols_data, rows_data, cols, rows, x0_index)
output

Switch to Phase 2.


,x1,x2,x3,w1,w2,HeSo
0,0.0,0.0,-0.666667,3.333333,2.666667,-4.0
1,0.0,1.0,0.333333,-0.666667,-0.333333,1.0
2,1.0,0.0,0.666667,-0.333333,0.333333,1.0


# Pha 2:

def two_Phase(data):

    #Lưu các thông số dòng và cột của data
    rows_data, cols_data = data.shape
    matrix_data = np.array(data, dtype=float)

    #Đưa về bài toán bổ trợ
    df = SupplementaryMath(data)

    #Lưu các thông số dòng và cột của df
    rows, cols = df.shape
    matrix = np.array(df, dtype=float)
    
    #Pha 1
    x0_index = Find_x0_Index(df)
    check, output = Phase1(data, matrix_data, df, matrix, rows, cols, x0_index)
    #Nếu Pha 1 trả về check == 2
    if check == 2:
        array = []
        for i in range(np.shape(output)[0]):
            array.append([str(output.values[i,j]) for j in range(np.shape(output)[1])])
        check, output = dantzig(array)
    return check, output